In [1]:
import os
from pyspark.sql import SparkSession

# Set environment vars to load jars
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--jars jars/hadoop-azure-3.3.6.jar," # Hadoop connector (wasb + wasbs)
    "jars/azure-storage-8.6.6.jar," # Azure SDK for Java (allow communication between Hadoop, Spark with Blobs Storage)
    "jars/jetty-client-9.4.43.v20210629.jar," # I don't know...
    "jars/jetty-http-9.4.43.v20210629.jar," ###############
    "jars/jetty-io-9.4.43.v20210629.jar," #################
    "jars/jetty-util-9.4.43.v20210629.jar," ################
    "jars/jetty-util-ajax-9.4.43.v20210629.jar " ############
    "pyspark-shell"
)


In [2]:
# Init spark session
spark = SparkSession.builder \
    .appName("Bronze to Silver: Table filter") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/07 13:45:40 WARN Utils: Your hostname, lenovo-slim, resolves to a loopback address: 127.0.1.1; using 192.168.199.13 instead (on interface wlp2s0)
25/07/07 13:45:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/07/07 13:45:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# Bronze access key
spark.conf.set(
    "fs.azure.account.key.mybronze.blob.core.windows.net",
    "c5etqTidViezB/4ukOAALy23HeMBsJJ8g+2nFaIdbC7E9PhLw0y2YIA1ItjutpqS1/8Ga8fw40mR+ASt2T+/sw=="
)

# Silver access key
spark.conf.set(
    "fs.azure.account.key.mysilver.dfs.core.windows.net",
    "bAthp0pVBfqEtyCvJElSX7MeI7ejSLa6cjuPoMz0Gg/69uzEW01y4URMDXsdFCrkpc9M54cDHnXs+AStj1gExQ=="
)

In [4]:
# Test
df = spark.read.parquet("wasbs://bronze@mybronze.blob.core.windows.net/Payments")

25/07/07 13:45:55 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-azure-file-system.properties,hadoop-metrics2.properties


In [5]:
# Test
df.show()

+---------+-------+---------------+------+-------------------+-------------------+
|PaymentID|OrderID|PaymentMethodID|Amount|          CreatedAt|          UpdatedAt|
+---------+-------+---------------+------+-------------------+-------------------+
|        1|      2|              5|   504|2024-09-23 06:38:26|2025-03-03 12:07:14|
|        2|      3|              4|   431|2024-11-16 01:10:46|2025-02-02 13:18:55|
|        3|      4|              4|   854|2025-04-07 00:14:45|2025-06-10 22:13:02|
|        4|      5|              5|   542|2025-01-15 17:20:21|2025-02-07 20:01:52|
|        5|      6|              5|   633|2024-12-29 21:55:54|2025-01-21 12:12:50|
|        6|      7|              3|   866|2025-02-11 04:47:16|2025-03-09 18:12:05|
|        7|      8|              5|   722|2025-03-30 01:26:14|2025-06-24 16:45:27|
|        8|      9|              2|   152|2024-12-13 04:29:01|2025-03-15 03:47:35|
|        9|     10|              5|   218|2024-08-26 05:37:02|2025-03-22 14:26:55|
|   

In [ ]:
df.write.mode("overwrite").parquet(f"abfss://silver@mysilver.dfs.core.windows.net/Payments")

In [ ]:
# Define storage account 
storage_account_bronze = "mybronze"
bronze_container = "bronze"
storage_account_silver = "mysilver"
silver_container = "silver"

tables = ["Customers", "Products", 
          "Sellers", "Orders", 
          "OrderItems","ProductCategories",
          "OrderStatus","Reasons", "Payments"]


In [11]:
for table in tables:
    bronze_path = f"wasbs://{bronze_container}@{storage_account_bronze}.blob.core.windows.net/{table}"
    silver_path = f"abfss://{silver_container}@{storage_account_silver}.dfs.core.windows.net/{table}"

    print(f"Processing {table}...")
    
    df = spark.read.parquet(bronze_path)
    df.write.mode("overwrite").parquet(silver_path)

    print(f"Written to {silver_path}")

    
    

Processing Customers...


Written to abfss://silver@mysilver.dfs.core.windows.net/Customers
Processing Products...


Written to abfss://silver@mysilver.dfs.core.windows.net/Products
Processing Sellers...


Written to abfss://silver@mysilver.dfs.core.windows.net/Sellers
Processing Orders...


Written to abfss://silver@mysilver.dfs.core.windows.net/Orders
Processing OrderItems...


Written to abfss://silver@mysilver.dfs.core.windows.net/OrderItems
Processing ProductCategories...


Written to abfss://silver@mysilver.dfs.core.windows.net/ProductCategories
Processing OrderStatus...


Written to abfss://silver@mysilver.dfs.core.windows.net/OrderStatus
Processing Reasons...


Written to abfss://silver@mysilver.dfs.core.windows.net/Reasons
